In [1]:
import pandas as pd 
import numpy as np
import scipy.stats as stats

In [13]:
df_sales = pd.read_csv('2022-04-01T12_df_sales.csv', parse_dates = ['date']).sort_values('date')
df_users = pd.read_csv('experiment_users.csv')



## Task 1

In [20]:
df_metrics = (
    df_sales
    .query("date >= '2022-03-16' & date < '2022-03-23'")
    .groupby('user_id', as_index=False)
    .agg(revenue = ('price', 'sum'))
)

df = pd.merge(df_users, df_metrics, on='user_id', how='left').fillna(0)

data_control = df[df['pilot'] == 0]['revenue']
data_pilot = df[df['pilot'] == 1]['revenue']
stats.ttest_ind(data_control, data_pilot)



Ttest_indResult(statistic=-1.2837567415000515, pvalue=0.19923983306424942)

## Task 2

Оцените среднее время между покупками. Возьмите всех клиентов, у которых 2 и более покупок. 

Вычислите время между покупками (для клиента с N покупками должно получиться N-1 значения времени). 

Объедините значения всех клиентов и вычислите среднее.

In [9]:
users_wth_orders = (
    df_sales
    .groupby('user_id', as_index=False)
    .agg(n_orders = ('sale_id', 'nunique'))
    .query("n_orders >= 2")
    .user_id
)

(
    df_sales
    .query("user_id in @users_wth_orders")
    .assign(
        diff_dt = (
            df_sales.query("user_id in @users_wth_orders").groupby('user_id').diff().date
        )
    )
    .diff_dt.mean()
)


Timedelta('17 days 07:37:57.653431857')

## Task 3

In [11]:
from pydantic import BaseModel

class User(BaseModel):
    login: str
    subscribed: bool = False
    age: int

user = User(login='winner2003', age='22', city='Moscow')
user

User(login='winner2003', subscribed=False, age=22)

## Task 4

In [12]:
import numpy as np
from pydantic import BaseModel
from scipy import stats


class Design(BaseModel):
    """Дата-класс с описание параметров эксперимента."""
    statistical_test: str


class ExperimentsService:

    def get_pvalue(self, metrics_a_group, metrics_b_group, design):
        """Применяет статтест, возвращает pvalue.

        :param metrics_a_group (np.array): массив значений метрик группы A
        :param metrics_a_group (np.array): массив значений метрик группы B
        :param design (Design): объект с данными, описывающий параметры эксперимента
        :return (float): значение p-value
        """
        if design.statistical_test == 'ttest':
            p_value = stats.ttest_ind(metrics_a_group, metrics_b_group, equal_var=False).pvalue
            return p_value
        else:
            raise ValueError('Неверный design.statistical_test')


if __name__ == '__main__':
    metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
    metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
    design = Design(statistical_test='ttest')
    ideal_pvalue = 0.612219

    experiments_service = ExperimentsService()
    pvalue = experiments_service.get_pvalue(metrics_a_group, metrics_b_group, design)
    np.testing.assert_almost_equal(ideal_pvalue, pvalue, decimal=4)
    print('simple test passed')

simple test passed
